In [1]:
import model_catalogs as mc
import intake
from tempfile import TemporaryDirectory
from pathlib import PurePath
import xarray as xr
import numpy as np

# Different Modes for Catalog Use

`model_catalogs`, as of v0.6.0, can use catalog files flexibly. This page demonstrates some of the uses.

You can install your catalog files or use local versions. Catalogs that are meant to be used long-term should be installed as Python packages. Once they are installed, they are available through the default `intake` catalog. See information on how to install your catalog files in the `docs page<add_model>`_.

See all available installed catalogs with:

In [2]:
list(intake.cat)

['mc_CBOFS',
 'mc_CBOFS_RGRID',
 'mc_CIOFS',
 'mc_CIOFS_RGRID',
 'mc_CREOFS',
 'mc_CREOFS_RGRID',
 'mc_DBOFS',
 'mc_DBOFS_RGRID',
 'mc_GFS_1DEG',
 'mc_GFS_1_2DEG',
 'mc_GFS_1_4DEG',
 'mc_GOFS',
 'mc_GOMOFS',
 'mc_GOMOFS_2DS',
 'mc_GOMOFS_RGRID',
 'mc_LEOFS',
 'mc_LEOFS_RGRID',
 'mc_LMHOFS',
 'mc_LMHOFS_RGRID',
 'mc_LOOFS',
 'mc_LOOFS_FVCOM',
 'mc_LSOFS',
 'mc_LSOFS_FVCOM',
 'mc_NGOFS2',
 'mc_NGOFS2_2DS',
 'mc_NGOFS2_RGRID',
 'mc_NYOFS',
 'mc_RTOFS',
 'mc_RTOFS_2D',
 'mc_RTOFS_AK',
 'mc_RTOFS_EAST',
 'mc_RTOFS_WEST',
 'mc_SFBOFS',
 'mc_SFBOFS_RGRID',
 'mc_TBOFS',
 'mc_TBOFS_RGRID',
 'mc_WCOFS',
 'mc_WCOFS_2DS',
 'mc_WCOFS_RGRID']

Installed catalogs to be used with `model_catalogs` should have a specified prefix ending with an underscore so they can be easily selected from the default catalog. `model_catalogs` required the installation of [`mc-goods`](https://github.com/axiom-data-science/mc-goods), a package of model catalogs, which have the prefix "mc_". 

# With or without `mc.setup()`

The default behavior is to read in the `mc-goods` catalogs, as follows

In [3]:
main_cat = mc.setup()  # default `locs="mc_"`
list(main_cat)

['CBOFS',
 'CBOFS_RGRID',
 'CIOFS',
 'CIOFS_RGRID',
 'CREOFS',
 'CREOFS_RGRID',
 'DBOFS',
 'DBOFS_RGRID',
 'GFS_1DEG',
 'GFS_1_2DEG',
 'GFS_1_4DEG',
 'GOFS',
 'GOMOFS',
 'GOMOFS_2DS',
 'GOMOFS_RGRID',
 'LEOFS',
 'LEOFS_RGRID',
 'LMHOFS',
 'LMHOFS_RGRID',
 'LOOFS',
 'LOOFS_FVCOM',
 'LSOFS',
 'LSOFS_FVCOM',
 'NGOFS2',
 'NGOFS2_2DS',
 'NGOFS2_RGRID',
 'NYOFS',
 'RTOFS',
 'RTOFS_2D',
 'RTOFS_AK',
 'RTOFS_EAST',
 'RTOFS_WEST',
 'SFBOFS',
 'SFBOFS_RGRID',
 'TBOFS',
 'TBOFS_RGRID',
 'WCOFS',
 'WCOFS_2DS',
 'WCOFS_RGRID']

If you have other catalogs available in the default catalog, you can instead input that prefix so the relevant catalogs can be found. For example, if the catalogs are installed with the prefix "other_", you can set up your main cat with them using

```
main_cat = mc.setup(locs="other_")
```

If you want to use those catalogs and the default catalogs, you can do so with

```
main_cat = mc.setup(locs=["mc_", "other_"])
```

You may instead set up with a local catalog file in which you input a string or Path of its location, for example,

```
main_cat = mc.setup(locs="path_to_catalog_file")
```

Or, you can input a combination of these.

For example, let's write a catalog file to demonstrate using a local catalog file:

In [4]:
with TemporaryDirectory() as tmpdirname:

    tmpdir = PurePath(tmpdirname)

    # Create a Dataset and save it
    dsfname = PurePath(tmpdir) / "example.nc"
    fname = PurePath(tmpdir) / 'example.yaml'

    ds = xr.Dataset()
    ds["time"] = ("Time", np.arange(10),)
    ds["Lat"] = ("y", np.arange(10),)
    ds["Lon"] = ("x", np.arange(10),)
    ds.to_netcdf(dsfname)

    catalog_text = f"""
    name: example
    metadata:
        alpha_shape: [4,50]  # dd, alpha
    sources:
        example_source:
            args:
                urlpath: {dsfname}
            description: ''
            driver: netcdf
            metadata:
                axis:
                    T: Time
                    X: x
                    Y: y
                standard_names:
                    time: time
                    latitude: Lat
                    longitude: Lon
                coords:
                    - Lon
                    - Lat
                    - Time
    """
    fp = open(fname, 'w')
    fp.write(catalog_text) 
    fp.close()

    main_cat = mc.setup(str(fname), override=True)
    print(list(main_cat["example"]))


['example_source']


Alternatively you can read in and work with a single catalog file with having the structure of nested catalogs in `mc.setup()`. For that, you can simply read in a catalog file with the following, and still have access to all the associated functions.

```
mc.open_catalog(catalog_path)
```